In [ ]:
directories = ['Project/2018','Project/2019','Project/2020' , 'Project/2021','Project/2022', 'Project/2023']

# Initialize a list to store the rows of the DataFrame
data_rows = []

# Columns to extract
columns_to_keep = [
    'coredata.dc:title',
    'coredata.prism:publicationName',
    'item.bibrecord.head.citation-info.citation-language.@language',
    'coredata.citedby-count'
]

# Helper function to safely extract a value from a dictionary
def safe_get(d, keys, default=None):
    try:
        for key in keys:
            d = d.get(key, {})
        return d if d else default
    except AttributeError:
        return default

# Iterate through each directory
for folder_path in directories:
    # Iterate through each file in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".json"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                try:
                    data = json.load(file)

                    # Extract countries from author-group
                    author_groups = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'author-group'], [])
                    countries = [
                        safe_get(author, ['affiliation', 'country'], 'Unknown')
                        for author in author_groups
                    ]
                    countries_string = ','.join(countries)

                    # Extract keywords
                    auth_keywords = safe_get(data, ['abstracts-retrieval-response', 'authkeywords', 'author-keyword'], [])
                    keywords = [keyword.get('$', '') for keyword in auth_keywords if isinstance(keyword, dict)]
                    keywords_string = ','.join(keywords)

                    # Normalize JSON data and filter columns
                    row_data = pd.json_normalize(data.get('abstracts-retrieval-response', {}))
                    row = {col: row_data[col].iloc[0] if col in row_data else pd.NA for col in columns_to_keep}

                    # Add processed fields
                    row['item.bibrecord.head.author-group.affiliation.country'] = countries_string
                    row['authkeywords.author-keyword'] = keywords_string

                    # Append the row to data_rows
                    data_rows.append(row)

                except Exception as e:
                    print(f"Error processing file {file_name} in {folder_path}: {e}")

# Convert the list of rows into a DataFrame
df = pd.DataFrame(data_rows)
# Save the DataFrame to a CSV file
output_path = 'outputAllYear.csv'
df.to_csv(output_path, index=False, encoding='utf-8')

# Display the DataFrame for verification
print(df)

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

# กรองข้อมูลที่ไม่ว่างเปล่า
result_df = result_df[result_df['authkeywords.author-keyword'].str.strip() != ""]

# แยกคำสำคัญทั้งหมดออกจากแถว
keywords = [keyword for row in result_df['authkeywords.author-keyword'].dropna() if isinstance(row, str) for keyword in row.split(',')]

# ใช้ TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_df=0.85, min_df=1, ngram_range=(1, 3))
X = vectorizer.fit_transform(keywords)

# ใช้ DBSCAN แทน KMeans
dbscan = DBSCAN(eps=0.5, min_samples=5)
labels = dbscan.fit_predict(X)

# คำนวณ Silhouette Score
silhouette_avg = silhouette_score(X, labels)
print("Silhouette Score with DBSCAN:", silhouette_avg)

In [ ]:
# Extract unique countries
unique_countries = set()
for countries in df['item.bibrecord.head.author-group.affiliation.country'].dropna():
    unique_countries.update(countries.split(','))

# Extract unique author keywords
unique_keywords = set()
for keywords in df['authkeywords.author-keyword'].dropna():
    unique_keywords.update(keyword for keyword in keywords.split(',') if keyword)

# Convert sets to lists for easier handling
unique_countries_list = list(unique_countries)
unique_keywords_list = list(unique_keywords)

# Display the unique countries and keywords
print("Unique Countries:", unique_countries_list)
print("Count of Unique Countries:", len(unique_countries_list))
print("\nUnique Author Keywords:", unique_keywords_list)
print("Count of Unique Author Keywords:", len(unique_keywords_list))

In [ ]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import os 
from sklearn.cluster import KMeans

# Define the path to the JSON file
json_file_path = 'Project/2023/202300000.json'

# Open and load the JSON file
with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Extract countries from author-group
countries = [author['affiliation']['country'] for author in data['abstracts-retrieval-response']['item']['bibrecord']['head']['author-group']]

# Convert the list of countries to a single string
countries_string = ','.join(countries)

# Extract $ values from authkeywords.author-keyword
keywords = [keyword['$'] for keyword in data['abstracts-retrieval-response']['authkeywords']['author-keyword']]

# Convert the list of keywords to a single string
keywords_string = ','.join(keywords)

# Normalize JSON data to a DataFrame
df = pd.json_normalize(data['abstracts-retrieval-response'])

# Define columns to keep
columns_to_keep = [
    'coredata.srctype',
    'coredata.eid',
    'coredata.dc:description',
    'coredata.pubmed-id',
    'coredata.prism:coverDate',
    'coredata.prism:aggregationType',
    'coredata.prism:url',
    'coredata.source-id',
    'coredata.pii',
    'coredata.citedby-count',
    'coredata.prism:volume',
    'coredata.subtype',
    'coredata.dc:title',
    'coredata.openaccess',
    'coredata.prism:issn',
    'coredata.publishercopyright',
    'coredata.article-number',
    'coredata.subtypeDescription',
    'coredata.prism:publicationName',
    'coredata.prism:doi',
    'coredata.dc:identifier',
    'coredata.dc:publisher',
    'item.bibrecord.head.citation-info.citation-language.@language',
]

# Add the countries_string and keywords_string to the DataFrame
df['item.bibrecord.head.author-group.affiliation.country'] = countries_string
df['authkeywords.author-keyword'] = keywords_string

# Keep only the specified columns
df = df[columns_to_keep + ['item.bibrecord.head.author-group.affiliation.country', 'authkeywords.author-keyword']]


df.to_csv('output.csv', index=False)

# Display the DataFrame to verify the contents
df.shape
df